In [0]:
!echo ======================================
!python --version
!echo ======================================
!echo OS
!cat /etc/issue.net
!echo ======================================
!echo CPU INFO
!head /proc/cpuinfo
!echo ======================================
!echo memory
!head -n 3 /proc/meminfo
!echo ======================================
!echo DISK
!df -h
!echo ======================================
!echo GPU
!nvidia-smi
!echo ======================================
!echo directory info
!mkdir google_drive
!ls -la

In [0]:
!pip install konlpy
!apt install -y -qq --fix-broken python-pycurl python-apt 2> /dev/null
!apt install -y -qq software-properties-common module-init-tools 2> /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt update -qq 2>&1 > /dev/null
!apt install -y -qq google-drive-ocamlfuse fuse 2> /dev/null

from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
creds = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

from getpass import getpass

vcode = getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p google_drive
!google-drive-ocamlfuse -o nonempty google_drive

In [0]:
! ls -l google_drive/datasets/nsmc

total 95587
drwxr-xr-x 2 root root     4096 May 28 12:27 code
-rw-r--r-- 1 root root  4893335 May 28 12:27 ratings_test.txt
-rw-r--r-- 1 root root 14628807 May 28 12:27 ratings_train.txt
-rw-r--r-- 1 root root 19515078 May 28 12:27 ratings.txt
drwxr-xr-x 2 root root     4096 May 28 12:27 raw
-rw-r--r-- 1 root root     2596 May 28 12:27 README.md
-rw-r--r-- 1 root root    36746 May 28 12:27 synopses.json
-rw-r--r-- 1 root root 14740402 May 28 13:05 test_docs.json
-rw-r--r-- 1 root root 44054253 May 28 13:05 train_docs.json


PreProcessing

In [0]:
cat google_drive/datasets/nsmc/ratings_train.txt | head -n 10

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


In [0]:
def read_data(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('google_drive/datasets/nsmc/ratings_train.txt')
test_data = read_data('google_drive/datasets/nsmc/ratings_test.txt')

데이터 불러오기 체크

In [0]:
print(len(train_data))
print(len(train_data[0]))
print(len(test_data))
print(len(test_data[0]))

150000
3
50000
3


KoNLPy & Okt

In [0]:
from konlpy.tag import Okt

okt = Okt()

In [0]:
import json
import os
from pprint import pprint

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('google_drive/datasets/nsmc/train_docs.json'):
    with open('google_drive/datasets/nsmc/train_docs.json') as f:
        train_docs = json.load(f)
    with open('google_drive/datasets/nsmc/test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('google_drive/datasets/nsmc/train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('google_drive/datasets/nsmc/test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

# 예쁘게(?) 출력하기 위해서 pprint 라이브러리 사용
pprint(train_docs[0])

[['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0']


In [0]:
import nltk

# 분석한 데이터의 토큰(문자열을 분석을 위한 작은 단위)의 갯수
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

# nltk를 이용한 전처리
text = nltk.Text(tokens, name='NMSC')
print(text)

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

2159921
<Text: NMSC>
2159921
49895
[('./Punctuation', 67778),
 ('영화/Noun', 50818),
 ('하다/Verb', 41209),
 ('이/Josa', 38540),
 ('보다/Verb', 38538),
 ('의/Josa', 30188),
 ('../Punctuation', 29055),
 ('가/Josa', 26627),
 ('에/Josa', 26468),
 ('을/Josa', 23118)]


In [0]:
selected_words = [f[0] for f in text.vocab().most_common(2048)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [0]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(2048,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

model_json = model.to_json()
with open("google_drive/app/model3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("google_drive/app/model_weights3.h5")
print("Saved model to drive")

Epoch 1/10
150000/150000 [==============================] - 2s 12us/sample - loss: 0.4094 - binary_accuracy: 0.8196
Epoch 2/10
150000/150000 [==============================] - 2s 11us/sample - loss: 0.3525 - binary_accuracy: 0.8418
Epoch 3/10
150000/150000 [==============================] - 2s 11us/sample - loss: 0.3349 - binary_accuracy: 0.8516
Epoch 4/10
150000/150000 [==============================] - 2s 11us/sample - loss: 0.3206 - binary_accuracy: 0.8612
Epoch 5/10
150000/150000 [==============================] - 2s 11us/sample - loss: 0.3074 - binary_accuracy: 0.8694
Epoch 6/10
150000/150000 [==============================] - 2s 11us/sample - loss: 0.2935 - binary_accuracy: 0.8762
Epoch 7/10
150000/150000 [==============================] - 2s 11us/sample - loss: 0.2803 - binary_accuracy: 0.8826
Epoch 8/10
150000/150000 [==============================] - 2s 12us/sample - loss: 0.2671 - binary_accuracy: 0.8891
Epoch 9/10
150000/150000 [==============================] - 2s 12us/samp

In [0]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰\n".format(review, (1 - score) * 100))

In [0]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")

[올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.]는 98.65% 확률로 긍정 리뷰

[배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.]는 71.34% 확률로 부정 리뷰

[주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ]는 90.69% 확률로 긍정 리뷰

[믿고 보는 감독이지만 이번에는 아니네요]는 50.73% 확률로 부정 리뷰

[주연배우 때문에 봤어요]는 74.27% 확률로 부정 리뷰



In [0]:
results

[0.38130987157821655, 0.8434]